In [1]:
import sys
import json
from fabric import Connection
from invoke import Responder, UnexpectedExit

sys.version

'3.8.10 (default, May 19 2021, 18:05:58) \n[GCC 7.3.0]'

In [ ]:
result = Connection('ubuntu@128.1.135.208').run('hostname', hide=False)

In [ ]:
result.stdout

In [2]:
hosts = [
    "118.193.45.223",
    "128.1.135.208",
    "103.14.34.147",
    "23.91.97.32",
    "152.32.210.117",
    "23.91.98.101",
    "152.32.189.174",
    "36.255.220.243",
    "152.32.192.130",
    "101.36.117.63",
    "101.36.120.12",
    "101.36.122.83",
    "152.32.215.147",
    "152.32.251.236",
    "152.32.190.173",
    "23.91.101.163",
    "118.193.39.223",
    "165.154.3.36",
    "45.249.247.180"
]

In [6]:
# Check status

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run("curl -s http://localhost:1635/peers | jq '.peers | length'", hide=True)
    print(f'peers = {result.stdout.strip()}')

==== 00/19 : 118.193.45.223 ====
swarm-00
peers = 259
==== 01/19 : 128.1.135.208 ====
swarm-01
peers = 290
==== 02/19 : 103.14.34.147 ====
swarm-02
peers = 276
==== 03/19 : 23.91.97.32 ====
swarm-03
peers = 224
==== 04/19 : 152.32.210.117 ====
swarm-04
peers = 288
==== 05/19 : 23.91.98.101 ====
swarm-05
peers = 243
==== 06/19 : 152.32.189.174 ====
swarm-06
peers = 272
==== 07/19 : 36.255.220.243 ====
swarm-07
peers = 294
==== 08/19 : 152.32.192.130 ====
swarm-08
peers = 218
==== 09/19 : 101.36.117.63 ====
swarm-09
peers = 274
==== 10/19 : 101.36.120.12 ====
swarm-10
peers = 309
==== 11/19 : 101.36.122.83 ====
swarm-11
peers = 247
==== 12/19 : 152.32.215.147 ====
swarm-12
peers = 307
==== 13/19 : 152.32.251.236 ====
swarm-13
peers = 301
==== 14/19 : 152.32.190.173 ====
10-7-155-240
peers = 0
==== 15/19 : 23.91.101.163 ====
10-7-36-8
peers = 0
==== 16/19 : 118.193.39.223 ====
10-7-70-72
peers = 0
==== 17/19 : 165.154.3.36 ====
10-7-166-180
peers = 0
==== 18/19 : 45.249.247.180 ====
10-7-

In [ ]:
# Check peer balances

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run('cd swarm-single-machine && ./balances.sh')
    print()

In [ ]:
# All down

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && ./down.sh')

In [ ]:
# All up

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && git pull')
    conn.run("cd swarm-single-machine && ./up.sh")

In [ ]:
# Rename hostname

sudopass = Responder(
    pattern=r'Password:',
    response='[please change]\n')

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run(f'hostnamectl set-hostname swarm-{i:02}', pty=True, watchers=[sudopass])
    conn.run('hostnamectl')
    print()

In [ ]:
# Add self hostname to /etc/hosts
# (to solve sudo error)

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('echo $(hostname -I | cut -d\  -f1) $(hostname) | sudo tee -a /etc/hosts')
    conn.run('cat /etc/hosts')

In [ ]:
# All addresses

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && ./addresses.sh')

In [5]:
# Print all eth addresses

for _, host in enumerate(hosts):
    conn = Connection(f'ubuntu@{host}')
    result = conn.run('cd swarm-single-machine && ./addresses.sh', hide=True)
    payload = json.loads(result.stdout.strip())
    print(payload['ethereum'])

0x19517c4542133b455a73368122a7a61ffcb53d33
0x84b134576e7841d3a77fddd629212273a3b2f45c
0x0a579b5e7d221668d48642387be24d0845aed796
0xf5afa5cde53355f8744596155668c7649a972407
0xb03a4dabaa26ec16164aa1aaf25b1144ad500a18
0xfe916470bb6e6d24d97259bcc53ae2e12b66a28b
0x1ff293414b41a76dcdbcb7b4d27124aa098e9638
0x1414ef73da8e3ca2f0306fe4b0f097cbfb57bd59
0x764a518c034f03855b4f97d2c60fbebc940b3a3c
0xd460d4a6364f008502d52bdc9aaea9fde645adfe
0xb25a5af6198c0157a4cc02bba9bb60a1589069e2
0x689e0900271bb7cdd42336a7b522036065269aa5
0x21d6fc3e662186c42aa3b30e1b911057a8639ebe
0xb3c39219a15d468133cb4b01347f0872f017c042
0xd9f2b033af099fac5f3931a23b5b33c9705a780e
0x02fca3269947cd84ae87527cc865706b6d180bb2
0x79855c3847c93ddcb2cc8d45607662737f6b2708
0x04490ee2ddbe105736256580e2a1c8f642e91324
0xf832b747b07d03f660097e75bc41bd6a51e11227


In [4]:
# Try to init all

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    try:
        conn.run('git clone https://github.com/renning22/swarm-single-machine.git && cd swarm-single-machine && ./init.sh')
    except UnexpectedExit:
        print('Skip...')

==== 00/19 : 118.193.45.223 ====
swarm-00


fatal: destination path 'swarm-single-machine' already exists and is not an empty directory.


Skip...
==== 01/19 : 128.1.135.208 ====
swarm-01


fatal: destination path 'swarm-single-machine' already exists and is not an empty directory.


Skip...
==== 02/19 : 103.14.34.147 ====
swarm-02


...

Creating network "swarm-single-machine_default" with the default driver
Pulling clef-1 (ethersphere/clef:0.4.12)...


0.4.12: Pulling from ethersphere/clef
Digest: sha256:3f6680ffbcad3c8cf9913d09add1d18be2caa4e8a8a89539ffd54c49de5a03e0
Status: Downloaded newer image for ethersphere/clef:0.4.12


Pulling bee-1 (ethersphere/bee:beta)...


beta: Pulling from ethersphere/bee
Digest: sha256:5738635ebc1be3e33438879c41c266ecea76dded59048fe842ef91f8935213b4
Status: Downloaded newer image for ethersphere/bee:beta


Creating swarm-single-machine_clef-1_1 ... 
ting swarm-single-machine_clef-1_1 ... doneCreating swarm-single-machine_bee-1_1  ... 
ting swarm-single-machine_bee-1_1  ... done

In [ ]:
# Restart unhealthy

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run("curl -s http://localhost:1635/peers | jq '.peers | length'", hide=True)
    print(f'peers = {result.stdout.strip()}')
    if not result.stdout.strip():
        print('Restarting...')
        conn.run('cd swarm-single-machine && ./restart.sh')